导入提示模板

In [1]:

from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import (
    ChatMessagePromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


使用PromptTemplate

In [2]:
from langchain import PromptTemplate

template = """\
你是业务咨询顾问。
你给一个销售{product}的电商公司，起一个好的名字？
"""
prompt = PromptTemplate.from_template(template)

print(prompt.format(product="鲜花"))


你是业务咨询顾问。
你给一个销售鲜花的电商公司，起一个好的名字？



In [3]:
prompt = PromptTemplate(
    input_variables=["product", "market"], 
    template="你是业务咨询顾问。对于一个面向{market}市场的，专注于销售{product}的公司，你会推荐哪个名字？"
)
print(prompt.format(product="鲜花", market="高端"))


你是业务咨询顾问。对于一个面向高端市场的，专注于销售鲜花的公司，你会推荐哪个名字？


# 使用ChatPromptTemplate,不同写法下：
## 第一种调用API的写法：

In [11]:
import os
os.environ["OPENAI_BASE_URL"] = 'https://chatapi.littlewheat.com/v1'
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model_name='gpt-3.5-turbo')
input = prompt.format(product="鲜花", market="高端")
#输出
output = model.invoke(input)

print(output.content)  


为一个面向高端市场、专注于销售鲜花的公司推荐名字时，可以考虑以下几个选项：

1. **花韵坊** - 强调花卉的优雅与韵味，适合高端市场。
2. **绽放精致** - 传达出鲜花的美丽与品质。
3. **花艺珍藏** - 突出鲜花的独特性和珍贵感。
4. **芳华时光** - 暗示鲜花与美好时光的结合，适合高端客户。
5. **奢花阁** - 直接传达高端奢华的感觉。
6. **花之奢品** - 强调鲜花作为一种奢侈品的定位。
7. **芬芳之巅** - 表达出高端、独特和品质的联想。

希望这些名字能帮助到你！


## 第二种调用

In [13]:
import os
from openai import OpenAI
os.environ["OPENAI_BASE_URL"] = 'https://chatapi.littlewheat.com/v1'
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"), 
)
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[ 
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

print(response.choices[0].message.content)


The 2020 World Series was played at Globe Life Field in Arlington, Texas. This was notable because it was the first time the World Series was held at a neutral site due to the COVID-19 pandemic.


## 第三种通过langchain调用方法

LangChain的ChatPromptTemplate模板示例

In [ ]:
# 导入聊天消息类模板
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
# 模板的构建
template="你是一位专业顾问，负责为专注于{product}的公司起名。"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="公司主打产品是{product_detail}。"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 格式化提示消息生成提示
prompt = prompt_template.format_prompt(product="鲜花装饰", product_detail="创新的鲜花设计。").to_messages()

# 下面调用模型，把提示传入模型，生成结果
import os
os.environ["OPENAI_BASE_URL"] = 'https://chatapi.littlewheat.com/v1'
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI()
result = chat(prompt)
print(result.content)


为您的鲜花装饰公司起名，可以考虑以下几个方向，以突出创新和独特性：

1. **花意独特**（Unique Bloom）
2. **绽放创想**（Blossom Innovations）
3. **花艺风尚**（Floral Flair）
4. **绚丽花境**（Vivid Floral Design）
5. **花海传奇**（Floral Odyssey）
6. **奇幻花园**（Enchanted Garden）
7. **花艺梦想**（Floral Dreams）
8. **绽放创意**（Creative Blossoms）
9. **色彩花语**（Colorful Petals）
10. **花开新意**（Fresh Bloom Concepts）

这些名字都强调了创新和美感，同时容易记忆，能够吸引客户的注意。希望您能找到一个合适的名字！


# 使用FewShotPromptTemplate

## 创建示例样本
    包含了四个字典，每个字典代表一种花的类型、迎合的场合、以及对应的广告文案，

In [1]:
# 1. 创建一些示例
samples = [
    {
    "flower_type": "玫瑰",
    "occasion": "爱情",
    "ad_copy": "玫瑰，浪漫的象征，是你向心爱的人表达爱意的最佳选择。"
    },
    {
    "flower_type": "康乃馨",
    "occasion": "母亲节",
    "ad_copy": "康乃馨代表着母爱的纯洁与伟大，是母亲节赠送给母亲的完美礼物。"
    },
    {
    "flower_type": "百合",
    "occasion": "庆祝",
    "ad_copy": "百合象征着纯洁与高雅，是你庆祝特殊时刻的理想选择。"
    },
    {
    "flower_type": "向日葵",
    "occasion": "鼓励",
    "ad_copy": "向日葵象征着坚韧和乐观，是你鼓励亲朋好友的最好方式。"
    }
]


## 创建提示模板

In [2]:
# 2. 创建一个提示模板
from langchain.prompts.prompt import PromptTemplate
template="鲜花类型: {flower_type}\n场合: {occasion}\n文案: {ad_copy}"
prompt_sample = PromptTemplate(
    input_variables=["flower_type", "occasion", "ad_copy"], 
    template=template
    )
print(prompt_sample.format(**samples[0]))


鲜花类型: 玫瑰
场合: 爱情
文案: 玫瑰，浪漫的象征，是你向心爱的人表达爱意的最佳选择。


## 创建FewShotPromptTemplate对象

In [3]:
# 3. 创建一个FewShotPromptTemplate对象
from langchain.prompts.few_shot import FewShotPromptTemplate
prompt = FewShotPromptTemplate(
    examples=samples,
    example_prompt=prompt_sample,
    suffix="鲜花类型: {flower_type}\n场合: {occasion}",
    input_variables=["flower_type", "occasion"]
)
print(prompt.format(flower_type="野玫瑰", occasion="爱情"))


鲜花类型: 玫瑰
场合: 爱情
文案: 玫瑰，浪漫的象征，是你向心爱的人表达爱意的最佳选择。

鲜花类型: 康乃馨
场合: 母亲节
文案: 康乃馨代表着母爱的纯洁与伟大，是母亲节赠送给母亲的完美礼物。

鲜花类型: 百合
场合: 庆祝
文案: 百合象征着纯洁与高雅，是你庆祝特殊时刻的理想选择。

鲜花类型: 向日葵
场合: 鼓励
文案: 向日葵象征着坚韧和乐观，是你鼓励亲朋好友的最好方式。

鲜花类型: 野玫瑰
场合: 爱情


## 调用大模型创新文案

In [4]:
# 4. 把提示传递给大模型
import os
os.environ["OPENAI_BASE_URL"] = 'https://chatapi.littlewheat.com/v1'
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model_name='gpt-3.5-turbo')
result = model(prompt.format(flower_type="野玫瑰", occasion="爱情"))
print(result)


C:\Users\34435\AppData\Local\Temp\ipykernel_34276\2579544378.py:6: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  result = model(prompt.format(flower_type="野玫瑰", occasion="爱情"))


content='文案: 野玫瑰，代表着自由与真爱的力量，是向心爱的人表达独特情感的绝佳选择。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 195, 'total_tokens': 226, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d54531d9eb', 'finish_reason': 'stop', 'logprobs': None} id='run-93337551-fb5d-42b0-a18b-61570016e674-0' usage_metadata={'input_tokens': 195, 'output_tokens': 31, 'total_tokens': 226}


## 使用示例选择器

In [8]:
# 5. 使用示例选择器
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

# 初始化示例选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
    samples,
    OpenAIEmbeddings(),
    Chroma,
    k=1
)

# 创建一个使用示例选择器的FewShotPromptTemplate对象
prompt = FewShotPromptTemplate(
    example_selector=example_selector, 
    example_prompt=prompt_sample, 
    suffix="鲜花类型: {flower_type}\n场合: {occasion}", 
    input_variables=["flower_type", "occasion"]
)
print(prompt.format(flower_type="红玫瑰", occasion="爱情"))


鲜花类型: 玫瑰
场合: 爱情
文案: 玫瑰，浪漫的象征，是你向心爱的人表达爱意的最佳选择。

鲜花类型: 红玫瑰
场合: 爱情
